In [1]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
import urllib.request
import os
#from pyquery import PyQuery as pq
import pickle
from selenium import webdriver
import datetime as dt
import time
import calendar
from scipy.stats.kde import gaussian_kde

In [2]:
#整理資料
origin_list=pd.read_csv("Materials Equity ETF List (62).csv",index_col=22,parse_dates=True)
origin_list2=pd.read_csv("Telecom Equity ETF List (10).csv",index_col=22,parse_dates=True)
need_list=pd.concat([origin_list,origin_list2])

In [3]:
#切出symbol欄&ETF名稱欄

symbol_list = need_list['Symbol'].reset_index()['Symbol']
name_list = need_list['ETF Name'].reset_index()['ETF Name']
all_ETF=need_list["Symbol"].values

In [4]:
#combine ETF daily data
#use the combined data to build the weekly and monthly ETF return 
def combine_daily_data(ETF):
    com_data=pd.DataFrame()
    for s in range(len(ETF)):
        read = "./data/"+ ETF[s] +".csv"
        data = pd.read_csv(read,index_col=0,parse_dates=True)["Adj Close"]
        com_data = pd.concat([com_data,data],axis=1)
    com_data.columns = ETF
    return com_data

Daily_ETF = combine_daily_data(all_ETF)

In [5]:
#fill in the no-trade dates
day=Daily_ETF.index[0]
n=0
while day!=Daily_ETF.index[-1]:
    if (Daily_ETF.index[n+1]-Daily_ETF.index[n]).days !=1:
        for i in range(1,(Daily_ETF.index[n+1]-Daily_ETF.index[n]).days):
            Daily_ETF.loc[Daily_ETF.index[n]+dt.timedelta(i)]=Daily_ETF.loc[Daily_ETF.index[n]]
    Daily_ETF=Daily_ETF.sort_index()
    n = n+(Daily_ETF.index[n+1]-Daily_ETF.index[n]).days
    day=Daily_ETF.index[n]
dirpath = os.getcwd() + '/completing missing data data'
filename = "Daily_ETF.csv"
Daily_ETF.to_csv(os.path.join(dirpath, filename))

In [6]:
week_index = []
for i in range(Daily_ETF.shape[0]):
    if Daily_ETF.index[i].isoweekday()==5:
        week_index.append(Daily_ETF.index[i])
        
#dirpath = os.getcwd() + '/completing missing data data'
#filename = "week_index.csv"
#week_index.to_csv(os.path.join(dirpath, filename))

month_index=[]
for i in range(Daily_ETF.shape[0]):
    if Daily_ETF.index[i].day==1:
        month_index.append(Daily_ETF.index[i])
        
#dirpath = os.getcwd() + '/completing missing data data'
#filename = "month_index.csv"
#month_index.to_csv(os.path.join(dirpath, filename))
#print(week_index)

In [7]:
riskless_rate=pd.read_csv("USG3M-new.csv",index_col=0,parse_dates=True)
USG3M=riskless_rate["COUPON EQUIVALENT"].reset_index("DATE")#['COUPON EQUIVALENT']
USG3M.set_index("DATE" , inplace=True)
n=0
day = USG3M.index[0]
print(day)
print(USG3M.index[-1])
while day!=USG3M.index[-1]:
    if (USG3M.index[n+1]-USG3M.index[n]).days !=1:
        for i in range(1,(USG3M.index[n+1]-USG3M.index[n]).days):
            USG3M.loc[USG3M.index[n]+dt.timedelta(i)]=USG3M.loc[USG3M.index[n]]
    USG3M=USG3M.sort_index()
    n = n+(USG3M.index[n+1]-USG3M.index[n]).days
    day=USG3M.index[n]
print(USG3M)
dirpath = os.getcwd() + '/completing missing data data'
filename = "USG3M.csv"
USG3M.to_csv(os.path.join(dirpath, filename))

2014-05-15 00:00:00
2019-05-15 00:00:00
            COUPON EQUIVALENT
DATE                         
2001-02-15               0.02
2001-02-18               1.44
2001-02-19               2.42
2001-03-17               0.54
2001-03-18               1.41
2001-03-19               2.41
2001-03-20               2.41
2001-03-21               2.41
2001-03-22               2.41
2001-03-23               2.41
2001-03-24               2.41
2001-03-25               2.41
2001-03-26               2.41
2001-03-27               2.41
2001-03-28               2.41
2001-03-29               2.41
2001-03-30               2.41
2001-03-31               2.41
2001-04-01               2.41
2001-04-02               2.41
2001-04-03               2.41
2001-04-04               2.41
2001-04-05               2.41
2001-04-06               2.41
2001-04-07               2.41
2001-04-08               2.41
2001-04-09               2.41
2001-04-10               2.41
2001-04-11               2.41
2001-04-12               2.41


In [8]:
dirpath = os.getcwd() + '/completing missing data data'

weekly_ETF=Daily_ETF.loc[week_index]
filename = 'Weekly_ETF.csv'
weekly_ETF.to_csv(os.path.join(dirpath, filename))

monthly_ETF=Daily_ETF.loc[month_index]
filename = 'Monthly_ETF.csv'
monthly_ETF.to_csv(os.path.join(dirpath, filename))

wr=(weekly_ETF/weekly_ETF.shift(1)-1 )
filename = 'wr.csv'
wr.to_csv(os.path.join(dirpath, filename))

mr=(monthly_ETF/monthly_ETF.shift(1)-1)
filename = 'mr.csv'
mr.to_csv(os.path.join(dirpath, filename))


wUSG3M = USG3M.loc[week_index] / 100 /52
filename = 'wUSG3M.csv'
wUSG3M.to_csv(os.path.join(dirpath, filename))

mUSG3M = USG3M.loc[month_index] / 100 /12
filename = 'mUSG3M.csv'
mUSG3M.to_csv(os.path.join(dirpath, filename))


0


/Users/jyun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


TypeError: 'COUPON EQUIVALENT    0.000154
Name: 2017-05-12 00:00:00, dtype: float64' is an invalid key